In [1]:
# dependencies: llama-index llama-index-llms-openai

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [ ]:
import phoenix as px
session = px.launch_app()

/Users/yujian/Documents/workspace/ai_agents_cookbooks/blog/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


Cannot query field 'id' on type 'SpanConnection'.

GraphQL request:61:11
60 |         descendants {
61 |           id
   |           ^
62 |           spanKind
Cannot query field 'spanKind' on type 'SpanConnection'.

GraphQL request:62:11
61 |           id
62 |           spanKind
   |           ^
63 |           name
Cannot query field 'name' on type 'SpanConnection'.

GraphQL request:63:11
62 |           spanKind
63 |           name
   |           ^
64 |           statusCode: propagatedStatusCode
Cannot query field 'propagatedStatusCode' on type 'SpanConnection'.

GraphQL request:64:11
63 |           name
64 |           statusCode: propagatedStatusCode
   |           ^
65 |           startTime
Cannot query field 'startTime' on type 'SpanConnection'.

GraphQL request:65:11
64 |           statusCode: propagatedStatusCode
65 |           startTime
   |           ^
66 |           latencyMs
Cannot query field 'latencyMs' on type 'SpanConnection'.

GraphQL request:66:11
65 |           startTim

In [57]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [3]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI


# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAI(model="gpt-4")

In [ ]:
input_docs = os.listdir("arize_blogs/")
input_docs

['agentic_rag.md',
 'multiagent_finetuning.md',
 'how_we_scaled_support_without_slowing_down.md',
 'unified_tool_for_evalution_and_observability.md',
 'agent_router_best_practices.md',
 'exploring_deepseek.md',
 'memory_and_state_in_llm_applications.md',
 'how_to_build_an_ai_agent.md']

In [5]:
doc_map = {}
for doc in input_docs:
    data = SimpleDirectoryReader(
        input_files=[f"./arize_blogs/{doc}"]
    ).load_data()
    doc_map[doc] = data

In [6]:
index_map = {}
for doc in doc_map:
    try:
        storage_context = StorageContext.from_defaults(
            persist_dir=f"./arize_blog_indexes/{doc}"
        )
        vector_index = load_index_from_storage(storage_context)
    except:
        vector_index = VectorStoreIndex.from_documents(doc_map[doc], show_progress=True)
    index_map[doc] = vector_index
    vector_index.storage_context.persist(persist_dir=f"./arize_blog_indexes/{doc}")

In [7]:
engines = {}
for index in index_map:
    engines[index] = index_map[index].as_query_engine(similarity_top_k=3, llm=llm)

In [41]:
engines

{'agentic_rag.md': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x1371d5fd0>,
 'multiagent_finetuning.md': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x135e8e030>,
 'how_we_scaled_support_without_slowing_down.md': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x1373c9cd0>,
 'unified_tool_for_evalution_and_observability.md': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x1373bc650>,
 'agent_router_best_practices.md': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x1373bc050>,
 'exploring_deepseek.md': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x1373d2f30>,
 'memory_and_state_in_llm_applications.md': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x1373d3050>,
 'how_to_build_an_ai_agent.md': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryE

In [9]:
from llama_index.core.llms import ChatMessage

In [38]:
with open("./arize_blogs/agentic_rag.md", "r") as f:
    lines = f.read()
messages = [
    ChatMessage(role="system", content="You are a helpful assistant. You describe the information the user provides and summarize it into two concise sentences."),
    ChatMessage(role="user", content=lines),
]
response = llm.chat(messages)
response

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The article discusses the concept of Agentic Retrieval-Augmented Generation (RAG), an advancement in AI applications that introduces intelligent agents into the retrieval process, enhancing the handling of complex queries across multiple data sources. It provides a detailed guide on implementing Agentic RAG, emphasizing the importance of clear tool descriptions, robust testing, document quality, and a comprehensive monitoring strategy for system performance.')]), raw=ChatCompletion(id='chatcmpl-BBTyZvVNbsLQ4XwcBTGLSmFOu0KKU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The article discusses the concept of Agentic Retrieval-Augmented Generation (RAG), an advancement in AI applications that introduces intelligent agents into the retrieval process, enhancing the handling of complex queries acro

In [39]:
from pprint import pprint

pprint(response.message.blocks[0].text)

('The article discusses the concept of Agentic Retrieval-Augmented Generation '
 '(RAG), an advancement in AI applications that introduces intelligent agents '
 'into the retrieval process, enhancing the handling of complex queries across '
 'multiple data sources. It provides a detailed guide on implementing Agentic '
 'RAG, emphasizing the importance of clear tool descriptions, robust testing, '
 'document quality, and a comprehensive monitoring strategy for system '
 'performance.')


In [23]:
# QueryEngineTool(
#         query_engine=lyft_engine,
#         metadata=ToolMetadata(
#             name="lyft_10k",
#             description=(
#                 "Provides information about Lyft financials for year 2021. "
#                 "Use a detailed plain text question as input to the tool."
#             ),
#         ),
#     ),

tools = []
for engine in engines:
    with open(f"./arize_blogs/{engine}", "r") as f:
        lines = f.read()
    messages = [
        ChatMessage(role="system", content="You are a helpful assistant. You describe the information the user provides and summarize it into two concise sentences."),
        ChatMessage(role="user", content=lines),
    ]
    response = llm.chat(messages)
    qetool = QueryEngineTool(
        query_engine = engines[engine],
        metadata=ToolMetadata(
            name=engine,
            description=(
                f"{response.message.blocks[0].text}."
                "Use a detailed plain text question as input to the"
            )
        )
    )
    tools.append(qetool)

In [45]:
descriptions = {}
for tool in tools:
    print(tool.metadata.description)
    print(tool.metadata.name)
    descriptions[tool.metadata.name] = tool.metadata.description

The article discusses the concept of Agentic Retrieval-Augmented Generation (RAG), an advanced AI application that introduces intelligent agents into the retrieval process to handle complex queries across multiple data sources. It provides a tutorial on building an Agentic RAG system, discusses its implementation, and emphasizes the importance of monitoring and improving its performance using tools like Arize Phoenix..Use a detailed plain text question as input to the
agentic_rag.md
The blog post discusses a conversation with Yilun Du, a Senior Research Scientist at Google DeepMind, about his latest research paper on a multiagent finetuning framework that improves the performance and diversity of language models. The framework uses a team of specialized models that learn from each other, maintaining diversity in reasoning and sustaining long-term performance gains, and has potential applications in various reasoning tasks and future advancements in language model development..Use a det

In [46]:
descriptions

{'agentic_rag.md': 'The article discusses the concept of Agentic Retrieval-Augmented Generation (RAG), an advanced AI application that introduces intelligent agents into the retrieval process to handle complex queries across multiple data sources. It provides a tutorial on building an Agentic RAG system, discusses its implementation, and emphasizes the importance of monitoring and improving its performance using tools like Arize Phoenix..Use a detailed plain text question as input to the',
 'multiagent_finetuning.md': 'The blog post discusses a conversation with Yilun Du, a Senior Research Scientist at Google DeepMind, about his latest research paper on a multiagent finetuning framework that improves the performance and diversity of language models. The framework uses a team of specialized models that learn from each other, maintaining diversity in reasoning and sustaining long-term performance gains, and has potential applications in various reasoning tasks and future advancements in 

In [47]:
import json
json_desc = json.dumps(descriptions, indent=4)

In [48]:
with open("descriptions.json", "w") as f:
    f.write(json_desc)

In [51]:
agent = ReActAgent.from_tools(
    tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

In [ ]:
prompt = """
What tool should I use for AI quality?

<hint> ALWAYS insert the URL to where you found the information in your response if you see it </hint>
"""

response = agent.chat(prompt)
print(str(response))

> Running step 155a9af0-8f74-4943-8beb-908abda7a0ff. Step input: 
What tool should I use for AI quality?

<hint> ALWAYS insert the URL to where you found the information in your response if you see it </hint>

Thought: The user is asking for a tool for AI quality. The tool "unified_tool_for_evalution_and_observability.md" seems to be the most relevant to the user's question as it discusses a platform for AI evaluation and observability. I will use this tool to provide a more detailed answer.
Action: unified_tool_for_evalution_and_observability.md
Action Input: {'input': 'What tool should I use for AI quality?'}
Observation: You should consider using Arize. It's a unified AI observability and evaluation platform designed specifically for AI engineers. It provides end-to-end observability, evaluation, and troubleshooting capabilities across all AI model types. This allows you to develop, evaluate, monitor, and iterate your AI models without silos or blind spots. Arize supports GenAI, Com

In [10]:
sources = {}
for engine in engines:
    with open(f"./arize_blogs/{engine}", "r") as f:
        lines = f.read()
    messages = [
        ChatMessage(role="system", content="""You are a helpful assistant. Extract and return the title and URL to the source .md file.
                    <hint> Always give answers in the form of
                    Title: (title)
                    URL: (url) </hint>"""),
        ChatMessage(role="user", content=lines),
    ]
    response = llm.chat(messages)
    title = response.message.blocks[0].text.split("\n")[0].split(": ")[1]
    url = response.message.blocks[0].text.split("\n")[1].split(": ")[1]

    sources[engine] = {"title": title,
                       "url": url}

In [64]:
response.message.blocks[0].text.split("\n")
title = response.message.blocks[0].text.split("\n")[0].split(": ")[1]
url = response.message.blocks[0].text.split("\n")[1].split(": ")[1]
print(title, url)

Understanding Agentic RAG http://arize.com/blog/understanding-agentic-rag/


In [11]:
sources

{'agentic_rag.md': {'title': 'Understanding Agentic RAG',
  'url': 'http://arize.com/blog/understanding-agentic-rag/'},
 'multiagent_finetuning.md': {'title': 'Multiagent Finetuning',
  'url': 'http://arize.com/blog/multiagent-finetuning-a-conversation-with-researcher-yilun-du/'},
 'how_we_scaled_support_without_slowing_down.md': {'title': 'How We Scaled Support in Arize Copilot Without Slowing Down',
  'url': 'http://arize.com/blog/how-we-scaled-support-in-arize-copilot-without-slowing-down/'},
 'unified_tool_for_evalution_and_observability.md': {'title': 'Why AI Engineers Need a Unified Tool for AI Evaluation and Observability',
  'url': 'http://arize.com/blog/why-ai-engineers-need-a-unified-tool-for-ai-evaluation-and-observability/'},
 'agent_router_best_practices.md': {'title': 'Best Practices for Building an Agent Router',
  'url': 'http://arize.com/blog/best-practices-for-building-an-ai-agent-router/'},
 'exploring_deepseek.md': {'title': 'How DeepSeek is Pushing the Boundaries o

In [12]:
import json
json_desc = json.dumps(sources, indent=4)
with open("sources.json", "w") as f:
    f.write(json_desc)

In [16]:
with open("./sources.json", "r") as f:
    data = json.load(f)

data
    

{'agentic_rag.md': {'title': 'Understanding Agentic RAG',
  'url': 'http://arize.com/blog/understanding-agentic-rag/'},
 'multiagent_finetuning.md': {'title': 'Multiagent Finetuning',
  'url': 'http://arize.com/blog/multiagent-finetuning-a-conversation-with-researcher-yilun-du/'},
 'how_we_scaled_support_without_slowing_down.md': {'title': 'How We Scaled Support in Arize Copilot Without Slowing Down',
  'url': 'http://arize.com/blog/how-we-scaled-support-in-arize-copilot-without-slowing-down/'},
 'unified_tool_for_evalution_and_observability.md': {'title': 'Why AI Engineers Need a Unified Tool for AI Evaluation and Observability',
  'url': 'http://arize.com/blog/why-ai-engineers-need-a-unified-tool-for-ai-evaluation-and-observability/'},
 'agent_router_best_practices.md': {'title': 'Best Practices for Building an Agent Router',
  'url': 'http://arize.com/blog/best-practices-for-building-an-ai-agent-router/'},
 'exploring_deepseek.md': {'title': 'How DeepSeek is Pushing the Boundaries o

In [17]:
data['agentic_rag.md']

{'title': 'Understanding Agentic RAG',
 'url': 'http://arize.com/blog/understanding-agentic-rag/'}

In [ ]:
for engine in engines:
    with open("descriptions.json", "r") as f:
        data = json.load(f)
    description = data[engine]
    qetool = QueryEngineTool(
            query_engine = engines[engine],
            metadata=ToolMetadata(
                name=engine,
                description=(
                    f"{description}."
                    "Use a detailed plain text question as input to the"
                )
            )
        )

TypeError: ToolMetadata.__init__() got an unexpected keyword argument 'title'